# Proof of Concept - ML with Homomorphic Encryption 

Basic idea is to try out very simple neural network in Torch to be converted into an equivalent network that processes homomorphic encrpyted data. 
In this example, we are using two libraries: TenSEAL by OpenMined and Concrete-ML by Zama. 

**Dataset used**: Titanic

**Model architecture**:
- Linear (256, 15) // 256 neurons, 15 input features
- ReLU 
- Linear (2, 256)  // 2 output class

--- 


Version Log and Todo:

- Need to recheck on Tenseal context to perform multiple matrix multiplications
---

This Proof of Concept is largely influenced by 

https://github.com/OpenMined/TenSEAL/blob/main/tutorials/Tutorial%201%20-%20Training%20and%20Evaluation%20of%20Logistic%20Regression%20on%20Encrypted%20Data.ipynb#:~:text=Tutorial,-1%20%2D%20Training%20and

In [1]:
import torch
import torch.nn as nn
import tenseal as ts
import pandas as pd
import random
from time import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# those are optional and are not necessary for training
import numpy as np
import matplotlib.pyplot as plt

# Dataset preparation

In [2]:
df = pd.read_csv(".data/titanic/train.csv")

In [3]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df, train=True):
        
        cdf = df.drop(['PassengerId', 'Name', 'Ticket'], axis=1)
        # bucketing
        cdf['Age'] = pd.cut(cdf['Age'], bins=[0, 18, 40, 60, 100],
                            labels=['child', 'young', 'middle', 'old'])

        # separate Cabin values
        def preprocess_cabin(x):
            if pd.isnull(x):
                return pd.Series([np.nan, np.nan, np.nan], 
                                index=['Number_of_rooms', 'Cabin_Deck', 'Cabin_Room'])
            else:
                rooms = x.split(' ')
                return pd.Series([len(rooms), rooms[0][0], rooms[0][1:]], 
                                index=['Number_of_rooms', 'Cabin_Deck', 'Cabin_Room'])

        # Apply preprocess_cabin and create new columns
        # cabin_features = cdf['Cabin'].apply(preprocess_cabin)
        # cdf[['Number_of_rooms', 'Cabin_Deck', 'Cabin_Room']] = cabin_features

        # one hot encoding
        cdf['Pclass'] = cdf['Pclass'].apply(
            lambda x: {1: 'upper', 2: 'middle', 3: 'lower'}[x])
        cdf = pd.get_dummies(
            cdf, columns=['Age', 'Embarked', 'Pclass', 'Sex'])

        cdf = cdf.drop(['Cabin',], axis=1)

        # First, separate numeric and boolean columns
        numeric_cols = ['SibSp', 'Parch', 'Fare']
        bool_cols = [col for col in cdf.columns if cdf[col].dtype == 'bool']

        # For numeric columns, use StandardScaler
        scaler = StandardScaler()
        cdf[numeric_cols] = scaler.fit_transform(cdf[numeric_cols])

        # Boolean columns are already 0 and 1, no need to normalize
        # But convert to float for PyTorch
        for col in bool_cols:
            cdf[col] = cdf[col].astype('float32')
        
        
        
        if train:    
            self.data = cdf.drop('Survived', axis=1).values.astype('float32')
            self.target = cdf['Survived'].values
        else:
            self.data = cdf.values.astype('float32')
            self.target = None
        self.nfeatures = self.data.shape[1]
        self.train = train
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.train:
            return self.data[idx], self.target[idx]
        else:
            return self.data[idx], np.nan
    
    

In [4]:
dataset = Dataset(df)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [5]:
test_dataset = Dataset(pd.read_csv(".data/titanic/test.csv"), train=False)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

# Model definition

In [6]:
class SimpleClassifier(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(), # skip ReLU to make life a little bit easier for now
                # but apparently, Zama's Concrete won't accept Linear combined together without activation? so turning this on
            nn.Linear(256, 2),
        )
        
    def forward(self, x):
        return self.model(x)

In [7]:
# no need for mps / cuda as its a very simple network
device = torch.device("cpu")

In [8]:
classifier = SimpleClassifier(dataset.nfeatures).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters())

In [9]:
for epoch in range(5):
    running_correct = 0
    running_total = 0
    
    classifier.train()
    
    for i, (data, target) in enumerate((dataloader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        
        output = classifier(data)
        
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        pred = output.argmax(dim=1)
        running_correct += (pred == target).sum().item()
        running_total += target.size(0)
    
    print(f"epoch: {epoch+1}")
    print(f"  loss: {loss.item()}")
    print(f"  accuracy: {running_correct / running_total}")

epoch: 1
  loss: 0.5248889327049255
  accuracy: 0.6734006734006734
epoch: 2
  loss: 0.48460689187049866
  accuracy: 0.7867564534231201
epoch: 3
  loss: 0.5914096832275391
  accuracy: 0.8103254769921436
epoch: 4
  loss: 0.37213996052742004
  accuracy: 0.8080808080808081
epoch: 5
  loss: 0.2948571443557739
  accuracy: 0.813692480359147


---

# Concrete ML

In [10]:
from concrete.ml.torch.compile import compile_torch_model
import numpy

torch_input = torch.randn(100, 15)
quantized_module = compile_torch_model(
    classifier.to('cpu'), # our model
    torch_input, # a representative input-set to be used for both quantization and compilation
    n_bits=6,
    rounding_threshold_bits={"n_bits": 6, "method": "approximate"}
)

quantized_module

In [11]:
data, target = next(iter(testloader))
x_test_q = quantized_module.quantize_input(data.numpy())
y_pred = quantized_module.quantized_forward(x_test_q, fhe="execute")
y_pred

array([[ 5436, -4608]])

In [12]:
y_pred = quantized_module.dequantize_output(y_pred)
y_pred

array([[ 1.00673574, -0.78111645]])

In [13]:
classifier(data)

tensor([[ 0.9749, -0.7431]], grad_fn=<AddmmBackward0>)

Still exploring to see how we could "examine" the encryption on Concrete (i.e. how we see the encrypted X and Y)

In [14]:
# for i, (data, target) in enumerate(testloader):
#     y_pred = quantized_module.forward(data.numpy(), fhe="execute")
#     print(y_pred)
#     break 

# Tenseal

In [16]:
classifier.model[0].weight.data.shape

torch.Size([256, 15])

In [31]:
# Create Tenseal Context 

# parameters
poly_mod_degree = 16384
coeff_mod_bit_sizes = [60, 40, 40, 60]
# create TenSEALContext
ctx_eval = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
# scale of ciphertext to use
ctx_eval.global_scale = 2 ** 40
# this key is needed for doing dot-product operations
ctx_eval.generate_galois_keys()


# # parameters
# poly_mod_degree = 16384  # Increased for more precision
# coeff_mod_bit_sizes = [50, 30, 30, 30, 50]  # More conservative chain
# scale = 2 ** 30  # Matched with middle layers

# # create TenSEALContext
# ctx_eval = ts.context(
#     scheme=ts.SCHEME_TYPE.CKKS,
#     poly_modulus_degree=poly_mod_degree,
#     coeff_mod_bit_sizes=coeff_mod_bit_sizes
# )
# ctx_eval.global_scale = scale
# ctx_eval.generate_galois_keys()

# ctx_eval = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree = 16384, coeff_mod_bit_sizes = [60, 60, 60, 60, 60, 60])
# ctx_eval.generate_galois_keys()
# ctx_eval.global_scale = 2**25

In [32]:
# Testing 
a = torch.rand((1, 5))
b = torch.ones((5, 2))

enc_a = ts.ckks_tensor(ctx_eval, a)
enc_b = ts.ckks_tensor(ctx_eval, b)

(a @ b)

tensor([[3.8791, 3.8791]])

In [33]:
torch.tensor((enc_a @ enc_b).decrypt().tolist())

tensor([[3.8792, 3.8792]])

In [34]:
print(type(enc_a))
print(type(b))
torch.tensor((enc_a @ b).decrypt().tolist())

<class 'tenseal.tensors.ckkstensor.CKKSTensor'>
<class 'torch.Tensor'>


tensor([[3.8792, 3.8792]])

In [35]:
x, _ = next(iter(testloader))
enc_x = ts.ckks_tensor(ctx_eval, x.tolist())

print(x)
print(type(enc_x))
print(type(classifier.model[0].weight.data))
enc_x @ (classifier.model[0].weight.data.cpu().T)

tensor([[-0.4995, -0.4002, -0.4978,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
          1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000]])
<class 'tenseal.tensors.ckkstensor.CKKSTensor'>
<class 'torch.Tensor'>


In [36]:
enc_x.scale

<bound method CKKSTensor.scale of <tenseal.tensors.ckkstensor.CKKSTensor object at 0x31c817e80>>

Simulate a forward pass - 

In [37]:
w_ = classifier.model[0].weight.data.cpu().T
w_.to(torch.float8_e4m3fn)
xl = enc_x @ (classifier.model[0].weight.data.cpu().T)

In [38]:
# Stuck at this point, scale out of bounds

w2_ = classifier.model[2].weight.data.cpu().T
w2_.to(torch.float8_e5m2)

xl @ (w2_)

In [ ]:
# Code in this cell and after did not consider RELU yet, so need to adapt (hardcoded 2 layers)
class EncryptedClassifier:
    
    def __init__(self, unencrypted_model):
        
        # assuming same architecture
        self.w1 = unencrypted_model.model[0].weight.data.cpu() # [256, 15]
        self.b1 = unencrypted_model.model[0].bias.data.cpu()  # [256]
        
        self.w2 = unencrypted_model.model[2].weight.data.cpu()  # [2, 256]
        self.b2 = unencrypted_model.model[2].bias.data.cpu()  # [2]
        

    # def forward(self, enc_x):
    #     # assume encrypted input
        
    #     # enc_x would be in the shape of [1, 15]
    #     # while w1 is in shape [256, 15] 
    #     results = [ ]
    #     for row in self.w1.T:
    #         part_result = enc_x @ row  # Dot product instead of full matrix mult
    #         results.append(part_result) # a little tricky here since the individual items are in form ckks_tensor, how to stack them (?)
        
    #     # Combine results
    #     x = torch.stack(results)
    #     x = enc_x @ self.w1.T 
    #     x = x @ self.w2.T 
        
    #     return x
 
 
    ## apparently, the context need to be adjusted so we can do matmul twice.
    ## it keeps making scale out of bounds error. To be researched
    def forward(self, enc_x):
        # assume encrypted input
        
        # enc_x would be in the shape of [1, 15]
        # while w1 is in shape [256, 15] 
        
        x = enc_x @ self.w1.T 
        # need relu, but not sure how
        x = x @ self.w2.T 
        
        return x        
        
        
        # enc_z = []
        
        # # calculate dot product of each neuron
        # for i in range(self.w1.shape[0]):               # 256 times
        #     # print(f"forward 1 - {i}")
        #     z = enc_x.dot(self.w1[i, :])    # dot product input x neuron weights
        #     z += self.b1[i]                 # add bias value 
        #     enc_z.append(z)         # add the result
            
        # # enc_z should be an array of [256, 1]
        # print(type(enc_z[0]))
        # print(type(enc_z))
        
        # # redo for second layer
        # enc_z2 = []
        # # calculate dot product of each neuron
        # for i in range(self.w2.shape[0]):   # 2 times
        #     print(f"forward 2 - {i}")
        #     z = enc_z.dot(self.w2[i, :])    # dot product input x neuron weights
        #     z += self.b2[i]                 # add bias value 
        #     enc_z2.append(enc_z2)         # add the result
                  
        # # enc_z should be an array of [2, 1]
        # return enc_z

    
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)
        

    
    def encrypt(self, context):
        self.w1 = ts.ckks_vector(context, self.w1)
        self.b1 = ts.ckks_vector(context, self.b1)
        
        
        self.w2 = ts.ckks_vector(context, self.w2)
        self.b2 = ts.ckks_vector(context, self.b2)
        
    def decrypt(self, context):
        self.w1 = self.w1.decrypt()
        self.b1 = self.b1.decrypt()
        
        self.w2 = self.w2.decrypt()
        self.b2 = self.b2.decrypt()
        

encCls = EncryptedClassifier(classifier)

In [44]:
data, target = next(iter(testloader))

In [50]:
for i, (data, target) in enumerate((testloader)):
    print(f"Instance {i}")
    
    x = data[0]
    print(f"\t input tensor: {x}")
    
    enc_x = ts.ckks_tensor(ctx_eval, [x])
    print(f"\t ckks enc tensor: {enc_x}")
    
    # encrypted evaluation
    enc_out = encCls(enc_x)
    print(f"\t forward output tensor: {enc_out}")
    
    # plain comparison
    out = enc_out.decrypt()
    print(f"\t decrypted output: {torch.tensor(out.tolist())}")
    
    if i == 5:
        break

Instance 0
	 input tensor: tensor([-0.4995, -0.4002, -0.4978,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
         1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000])
	 ckks enc tensor: <tenseal.tensors.ckkstensor.CKKSTensor object at 0x3c6f2a590>
	 forward output tensor: <tenseal.tensors.ckkstensor.CKKSTensor object at 0x3c6f29990>
	 decrypted output: tensor([[ 1.2319, -1.2118]])
Instance 1
	 input tensor: tensor([ 0.6170, -0.4002, -0.5127,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
         0.0000,  1.0000,  1.0000,  0.0000,  0.0000,  1.0000,  0.0000])
	 ckks enc tensor: <tenseal.tensors.ckkstensor.CKKSTensor object at 0x17899ead0>
	 forward output tensor: <tenseal.tensors.ckkstensor.CKKSTensor object at 0x3c6f2a140>
	 decrypted output: tensor([[-0.3896,  0.3020]])
Instance 2
	 input tensor: tensor([-0.4995, -0.4002, -0.4645,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,
         1.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000])
	 ckks enc tensor: <tenseal.t